In [4]:
import pandas as pd
import os
import holidays
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# Cargar el DataFrame desde el archivo CSV
data_path = os.path.join(os.getcwd(), 'data')  # Ruta al directorio 'data'
train = pd.read_csv(os.path.join(data_path, 'train.csv'))

# Asegurarse de que la columna 'date' esté en formato datetime
train['date'] = pd.to_datetime(train['date'], errors='coerce')

# Asegurarse que solo hay un registro por fecha y artículo
train = train.groupby(['date', 'item'], as_index=False)['sales'].sum()

# Eliminar filas con fechas inválidas
train = train.dropna(subset=['date'])

# Definir las categorías de ventas
bins = [0, 50, 100, float('inf')]
labels = ['bajo', 'medio', 'alto']
train['sales_category'] = pd.cut(train['sales'], bins=bins, labels=labels)

# Extraer características de tiempo adicionales
train['dayofweek'] = train['date'].dt.dayofweek
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train['dayofyear'] = train['date'].dt.dayofyear
train['is_weekend'] = train['date'].dt.dayofweek >= 5

# Agregar información sobre días festivos
us_holidays = holidays.MEX(years=[2013, 2014, 2015, 2016, 2017])
train['is_holiday'] = train['date'].isin(us_holidays)

# Convertir booleanos a enteros
train['is_weekend'] = train['is_weekend'].astype(int)
train['is_holiday'] = train['is_holiday'].astype(int)

# Crear variables dummy para la característica 'item'
train = pd.get_dummies(train, columns=['item'], drop_first=True)

# Codificar la característica de fecha como días desde una fecha de referencia
date_min = train['date'].min()
train['date'] = (train['date'] - date_min).dt.days

# Crear interacciones entre características
for col in train.columns:
    if col.startswith('item_'):
        train[f'date_{col}'] = train['date'] * train[col]

# Dividir los datos en conjuntos de entrenamiento y prueba
X = train.drop(columns=['sales', 'sales_category'])
y = train['sales_category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar un modelo de clasificación (RandomForestClassifier en este caso)
clf = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30]
}
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

best_clf = grid_search.best_estimator_

# Evaluar el modelo
y_pred = best_clf.predict(X_test_scaled)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Guardar el modelo en un best_clf.pkl y scaler en un scaler.pkl
import joblib
joblib.dump(best_clf, 'best_clf.pkl')
joblib.dump(scaler, 'scaler.pkl')


/tmp/ipykernel_23717/1663609162.py:36: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  train['is_holiday'] = train['date'].isin(us_holidays)


              precision    recall  f1-score   support

        alto       1.00      1.00      1.00     18243
       medio       0.43      0.18      0.25        17

    accuracy                           1.00     18260
   macro avg       0.71      0.59      0.62     18260
weighted avg       1.00      1.00      1.00     18260

[[18239     4]
 [   14     3]]


['scaler.pkl']

In [12]:
#cargar el train.csv
test = pd.read_csv(os.path.join(data_path, 'train.csv'))
#eliminar todos los datos que no del store 1 dejando el date,store,item y sales
test = test[test['store'] == 1]

#guardar el pd en un csv
test.to_csv(os.path.join(data_path, 'test_store1.csv'), index=False)

In [5]:

# Datos de prueba para predicción
test = [
  {'date': '2024-01-01', 'item': 1},
  {'date': '2024-01-01', 'item': 2},
  {'date': '2024-01-01', 'item': 3},
  {'date': '2024-01-01', 'item': 4},
  {'date': '2024-01-01', 'item': 5},
  {'date': '2024-01-01', 'item': 6},
  {'date': '2024-01-01', 'item': 7},
  {'date': '2024-01-01', 'item': 8},
  {'date': '2024-01-01', 'item': 9},
  {'date': '2016-02-14', 'item': 4}
]

# Convertir a DataFrame
test_df = pd.DataFrame(test)

# Asegurarse de que la columna 'date' esté en formato datetime
test_df['date'] = pd.to_datetime(test_df['date'], errors='coerce')

# Extraer características de tiempo adicionales
test_df['dayofweek'] = test_df['date'].dt.dayofweek
test_df['month'] = test_df['date'].dt.month
test_df['year'] = test_df['date'].dt.year
test_df['dayofyear'] = test_df['date'].dt.dayofyear
test_df['is_weekend'] = test_df['date'].dt.dayofweek >= 5

# Agregar información sobre días festivos
test_df['is_holiday'] = test_df['date'].isin(us_holidays)

# Convertir booleanos a enteros
test_df['is_weekend'] = test_df['is_weekend'].astype(int)
test_df['is_holiday'] = test_df['is_holiday'].astype(int)

# Crear variables dummy para la característica 'item'
test_df = pd.get_dummies(test_df, columns=['item'], drop_first=True)

# Codificar la característica de fecha como días desde una fecha de referencia
test_df['date'] = (test_df['date'] - date_min).dt.days

# Crear interacciones entre características
for col in test_df.columns:
    if col.startswith('item_'):
        test_df[f'date_{col}'] = test_df['date'] * test_df[col]

# Asegurarse de que las columnas en el conjunto de prueba coinciden con las del entrenamiento
expected_columns = set(X_train.columns)
current_columns = set(test_df.columns)
missing_columns = expected_columns - current_columns

for col in missing_columns:
    test_df[col] = 0

# Ordenar las columnas para que coincidan
test_df = test_df[X_train.columns]

# Escalar los datos de prueba
test_df_scaled = scaler.transform(test_df)

# Hacer predicciones con el mejor modelo encontrado
test_predictions = best_clf.predict(test_df_scaled)

# Mostrar las predicciones
print("Predicciones de categoría de ventas:", test_predictions)


Predicciones de categoría de ventas: ['alto' 'alto' 'alto' 'alto' 'alto' 'alto' 'alto' 'alto' 'alto' 'alto']


/tmp/ipykernel_23717/707317462.py:29: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  test_df['is_holiday'] = test_df['date'].isin(us_holidays)
